In [1]:
import tensorflow as tf
import time

/Users/midhunpk/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [87]:
from alpha import iterate_minibatches
from cnn_utils import  conv_layer, fc_layer

In [71]:
from utils import batch_all_triplet_loss
from utils import batch_hard_triplet_loss

In [141]:
from tqdm import tqdm
import os, cv2, random
import numpy as np

num_data = 500
IMG_SIZE = 99
data = []
labels = []
DIR = './train'
for directory in os.listdir(DIR):
    if not directory.startswith('.'):
        path1 = os.path.join(DIR, directory)
        for i, img in enumerate(os.listdir(path1)):
            if not img.startswith('.'):
                if i < num_data:
                    path2 = os.path.join(path1, img)
                    img = cv2.imread(path2)
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
                    labels.append([np.array(directory)])
                    data.append([np.array(img)])
#print("{0} Files loaded from each classes".format(i+1))
random.Random(1).shuffle(data)
random.Random(1).shuffle(labels)
train = np.array(data).reshape(-1,IMG_SIZE,IMG_SIZE,3)
label = np.array(labels)

In [142]:
def saveGraph(graph):
    with tf.Session(graph=graph) as sess:
        filename = "./summary_log/VS-"+time.strftime("%H%M%S")
        writer = tf.summary.FileWriter(filename, sess.graph)
    print("Tensorboard summary saved to "+filename) 

In [143]:
graph1 = tf.Graph()

In [144]:
with graph1.as_default():
    X = tf.placeholder(tf.float32, [None, 99, 99, 3])
    Y = tf.placeholder(tf.float32, [None, 1])

In [145]:
with graph1.as_default(): 
    conv1 = conv_layer(X,     3,  8,  "conv1")
    conv_out = conv_layer(conv1, 8,  16, "conv2") 
    flattened = tf.reshape(conv_out, [-1, 25*25*16])
    embeddings = fc_layer(flattened, 25*25*16, 128, "fc1")
    embedding_mean_norm = tf.reduce_mean(tf.norm(embeddings, axis=1))
    tf.summary.scalar("embedding_mean_norm", embedding_mean_norm)

In [147]:
with graph1.as_default(): 
    labels = tf.cast(Y, tf.int64)
    
    loss, _ = batch_all_triplet_loss(labels, embeddings, margin=2,
                                                squared=False)
    optimizer = tf.train.AdamOptimizer(learning_rate)
    global_step = tf.train.get_global_step()
    train_op = optimizer.minimize(loss, global_step=global_step)

In [139]:
saveGraph(graph1)

Tensorboard summary saved to ./summary_log/VS-004843


In [154]:
minibatch_size = 8
epoch = 10
learning_rate = 0.001

with tf.Session(graph=graph1) as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    saver.restore(sess, tf.train.latest_checkpoint('./saved_model/'))
    for epoch in range(epoch):
        avg_cost = 0
        for i, minibatch in enumerate(iterate_minibatches(train, label, minibatch_size)):
            (minibatch_X, minibatch_Y) = minibatch
            
            sess.run(train_op, feed_dict={X: minibatch_X, Y: minibatch_Y})
            
            avg_cost += sess.run(loss, feed_dict={X: minibatch_X, Y: minibatch_Y})

        print ("Cost after iteration  %f " % ( avg_cost/(i+1)))
    saver.save(sess, "./saved_model/model", global_step=global_step)

INFO:tensorflow:Restoring parameters from ./saved_model/model
Cost after iteration  1.946570 
Cost after iteration  1.852927 
Cost after iteration  1.887784 
Cost after iteration  1.879557 
Cost after iteration  1.831576 
Cost after iteration  1.896723 
Cost after iteration  1.925701 
Cost after iteration  1.880217 
Cost after iteration  1.782266 
Cost after iteration  1.822081 


In [163]:
data = pd.read_csv('./test/small-2oq-c1r.csv', nrows=2000)

In [164]:
data.head()

,productId,title,description,imageUrlStr,mrp,sellingPrice,specialPrice,productUrl,categories,productBrand,...,keySpecsStr,detailedSpecsStr,specificationList,sellerName,sellerAverageRating,sellerNoOfRatings,sellerNoOfReviews,sleeve,neck,idealFor
0,SPWE7T5EF93UGMZZ,Smilzo Saree Panty Slp/2808 Women's Shapewear,The wide and comfortable of Fashion Shapewears...,http://img.fkcdn.com/image/shapewear/x/7/7/slp...,799.0,798.0,798.0,http://dl.flipkart.com/dl/smilzo-saree-panty-s...,Apparels>Women>Lingerie & Sleepwear>Shapewears,Smilzo,...,Type: Shapewear Briefs;Women's Shapewear;Fabri...,Type: Shapewear Briefs;Women's Shapewear;Fabri...,NaN,SHAPING SYSTEMS INDIA PVT LTD,3.9,7,0,NaN,NaN,NaN
1,BLODYGFS6D4CZZDZ,Inblue Fashions Round Neck Women's Stitched Bl...,Inblue Fashions Black & White lace Silk Blouse...,http://img.fkcdn.com/image/blouse/q/z/m/inbblu...,3098.0,1550.0,1550.0,http://dl.flipkart.com/dl/inblue-fashions-roun...,Apparels>Women>Ethnic Wear>Blouses,Inblue Fashions,...,Round Neck;Fabric: Silk;Pattern: Floral Print,Round Neck;Fabric: Silk;Pattern: Floral Print,NaN,Inblue Fashions,4.2,23,1,NaN,Round Neck,NaN
2,BLODYGFSHBVBKJUK,Inblue Fashions Round Neck Women's Stitched Bl...,Inblue Fashions Black & White lace Silk Blouse...,http://img.fkcdn.com/image/blouse/q/z/m/inbblu...,3098.0,1549.0,1549.0,http://dl.flipkart.com/dl/inblue-fashions-roun...,Apparels>Women>Ethnic Wear>Blouses,Inblue Fashions,...,Round Neck;Fabric: Silk;Pattern: Floral Print,Round Neck;Fabric: Silk;Pattern: Floral Print,NaN,Inblue Fashions,4.2,23,1,NaN,Round Neck,NaN
3,BLODYGFSRTZAWHHE,Inblue Fashions Round Neck Women's Stitched Bl...,Inblue Fashions Black & White lace Silk Blouse...,http://img.fkcdn.com/image/blouse/q/z/m/inbblu...,3098.0,1549.0,1549.0,http://dl.flipkart.com/dl/inblue-fashions-roun...,Apparels>Women>Ethnic Wear>Blouses,Inblue Fashions,...,Round Neck;Fabric: Silk;Pattern: Floral Print,Round Neck;Fabric: Silk;Pattern: Floral Print,NaN,Inblue Fashions,4.2,23,1,NaN,Round Neck,NaN
4,BLODYGFSFARJXJB2,Inblue Fashions Round Neck Women's Stitched Bl...,Inblue Fashions Black & White lace Silk Blouse...,http://img.fkcdn.com/image/blouse/q/z/m/inbblu...,3098.0,1550.0,1550.0,http://dl.flipkart.com/dl/inblue-fashions-roun...,Apparels>Women>Ethnic Wear>Blouses,Inblue Fashions,...,Round Neck;Fabric: Silk;Pattern: Floral Print,Round Neck;Fabric: Silk;Pattern: Floral Print,NaN,Inblue Fashions,4.2,23,1,NaN,Round Neck,NaN


In [165]:
data.tail()

,productId,title,description,imageUrlStr,mrp,sellingPrice,specialPrice,productUrl,categories,productBrand,...,keySpecsStr,detailedSpecsStr,specificationList,sellerName,sellerAverageRating,sellerNoOfRatings,sellerNoOfReviews,sleeve,neck,idealFor
1995,SARDP7YVJB7W7PGG,"De Marca Solid Art Silk, Jacquard Sari(Multico...",Traditionally Printed Art Silk Saree.,http://img.fkcdn.com/image/sari/p/g/g/107b2-de...,3681.0,1399.0,1399.0,http://dl.flipkart.com/dl/de-marca-solid-art-s...,Apparels>Women>Ethnic Wear>Saris,De Marca,...,"Saree Fabric: Art Silk, Jacquard;With Blouse P...","Saree Fabric: Art Silk, Jacquard;With Blouse P...",NaN,Rubab Humza,3.900000,72,7,NaN,NaN,NaN
1996,SARDP7YVMPETMNMQ,De Marca Printed Chiffon Sari(Multicolor),Faux Chiffon Printed Sarees With Blouse Piece.,http://img.fkcdn.com/image/sari/n/m/q/c-30-de-...,2628.0,999.0,999.0,http://dl.flipkart.com/dl/de-marca-printed-chi...,Apparels>Women>Ethnic Wear>Saris,De Marca,...,Saree Fabric: Chiffon;With Blouse Piece,Saree Fabric: Chiffon;With Blouse Piece,NaN,Rubab Humza,3.900000,72,7,NaN,NaN,NaN
1997,SARE6CYW6QCZMFCE,Nilu Tex Self Design Daily Wear Handloom Georg...,NaN,http://img.fkcdn.com/image/sari/f/c/e/1-1-ntsh...,4100.0,2599.0,2599.0,http://dl.flipkart.com/dl/nilu-tex-self-design...,Apparels>Women>Ethnic Wear>Saris,Nilu Tex,...,Style: Sari;Saree Fabric: Georgette;Blouse Fab...,Style: Sari;Saree Fabric: Georgette;Blouse Fab...,NaN,NILU,0.000000,0,0,NaN,NaN,NaN
1998,SARE4DVYBR2TCC2G,Soni Sarees Printed Fashion Handloom Pure Geor...,"Multi color( Yellow, Blue, pink,green) with ab...",http://img.fkcdn.com/image/sari/c/2/g/ssp062-s...,599.0,399.0,399.0,http://dl.flipkart.com/dl/soni-sarees-printed-...,Apparels>Women>Ethnic Wear>Saris,Soni Sarees,...,With Blouse Piece;Fashion Sari;Pure Georgette;...,NaN,NaN,SSProducts,3.666667,6,2,NaN,NaN,NaN
1999,SARE43HF8CSVK5BH,Celebez Embroidered Fashion Handloom Satin Sar...,NaN,http://img.fkcdn.com/image/sari/5/b/h/celea016...,7800.0,3200.0,3200.0,http://dl.flipkart.com/dl/celebez-embroidered-...,Apparels>Women>Ethnic Wear>Saris,Celebez,...,Saree Fabric: Satin;Type: Fashion,Saree Fabric: Satin;Type: Fashion,NaN,sarfaraz ansari,3.000000,2,0,NaN,NaN,NaN


In [170]:
data.imageUrlStr[189]

'http://img.fkcdn.com/image/tunic/j/g/m/1-1-black-high-low-loose-fit-tunic-lastinch-m-original-imae7fr7vpqfpvr9.jpeg;http://img.fkcdn.com/image/tunic/j/g/m/1-1-black-high-low-loose-fit-tunic-lastinch-m-200x200-imae7fr7vpqfpvr9.jpeg;http://img.fkcdn.com/image/tunic/j/g/m/1-1-black-high-low-loose-fit-tunic-lastinch-m-400x400-imae7fr7vpqfpvr9.jpeg;http://img.fkcdn.com/image/tunic/j/g/m/1-1-black-high-low-loose-fit-tunic-lastinch-m-800x800-imae7fr7vpqfpvr9.jpeg'

In [130]:
import pandas as pd
import time

In [131]:
t1=time.time()
data = pd.read_csv('./../../Transfer_Learning/Bottle_neck_values.csv')
t2=time.time()

t2-t1

0.9812238216400146

In [125]:
data =np.array(data)

In [127]:
data.shape

(1200, 2048)

In [126]:
np.save('./../../Transfer_Learning/fnumpy.npy', data)

In [132]:
t1=time.time()
array_reloaded = np.load('./../../Transfer_Learning/fnumpy.npy').reshape(1200, 2048)
t2=time.time()
t2-t1

0.011577129364013672

In [133]:
5000*19/48

1979.1666666666667